# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [1]:
!pip install kaggle

In [2]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

ModuleNotFoundError: No module named 'google'

In [ ]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

In [ ]:
train_df.describe()

In [ ]:
from sklearn.linear_model import LinearRegression



linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

In [ ]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

In [ ]:
linear_reg.coef_

In [ ]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

In [ ]:
log_reg.predict(test_case)[0]

In [ ]:
help(log_reg.predict)

In [ ]:
log_reg.predict_proba(test_case)[0]

In [ ]:
# What's the math?
log_reg.coef_

In [ ]:
log_reg.intercept_

In [ ]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [ ]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [ ]:
# Live - let's try absenteeism!

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [4]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LogisticRegression

#!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

Archive:  fma_metadata.zip
   skipping: fma_metadata/README.txt  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/checksums  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/not_found.pickle  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/raw_genres.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/raw_albums.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/raw_artists.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/raw_tracks.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/tracks.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/genres.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/raw_echonest.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/echonest.csv  need PK compat. v4.6 (can do v4.5)
   skipping: fma_metadata/features.csv  need PK compat. v4.6 (can do v4.5)


In [5]:
tracks = pd.read_csv('edited_tracks.csv')

/anaconda2/envs/py3k/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,1,5,6,7,10,15,17,18,19,21,27,28,32,33,37,39,40,41,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
tracks.describe()

,track_id,comments,date_created,date_released,engineer,favorites,id,listens,producer,title,...,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94
count,136553,119347,111126,77948,18359,109238,109067,107855,19413,106404,...,35,35,35,35,35,35,35,35,35,35
unique,107169,298,13960,3768,671,113,15027,11889,790,14302,...,1,1,1,1,1,1,1,1,1,1
top,<p> </p>,0,Arial,1/1/08 0:00,Ernie Indradat,0,-1,-1,Joe Belock,microSong Entries,...,and aphorisms - currently runs into thousands...,for instance,Raw Dog Screaming Press (USA) and Blood Puddi...,Discharge,and contributes regularly to the on-line publ...,the title we just forgot,unRecordists,New Absurdist,and Translated Text of Automatic Form. He hol...,and is a recognised authority on aspects of C...
freq,5793,65003,4042,667,876,32104,825,1847,855,310,...,35,35,35,35,35,35,35,35,35,35


In [7]:
pd.set_option('display.max_columns', None)  # Unlimited columns
tracks.head()

,track_id,comments,date_created,date_released,engineer,favorites,id,listens,producer,title,tracks,type,active_year_begin,active_year_end,associated_labels,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,interest,language_code,listens.1,lyricist,number,publisher,title.1,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94
0,2,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,168,2,Hip-Hop,[21],[21],4656,en,1293,NaN,3,NaN,Food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,medium,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,237,1,Hip-Hop,[21],[21],1470,en,514,NaN,4,NaN,Electric Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,206,6,Hip-Hop,[21],[21],1933,en,1151,NaN,6,NaN,This World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0,11/26/08 1:45,2/6/08 0:00,NaN,4,6,47632,NaN,Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...",3,11/26/08 1:42,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,training,small,192000,0,Kurt Vile,11/25/08 17:49,11/26/08 0:00,161,178,Pop,[10],[10],54881,en,50135,NaN,1,NaN,Freeway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:48,1/1/08 0:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",978,en,361,NaN,3,NaN,Spiritual Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
tracks.shape

(136641, 95)

In [9]:
tracks = tracks[:45]
tracks.head()

,track_id,comments,date_created,date_released,engineer,favorites,id,listens,producer,title,tracks,type,active_year_begin,active_year_end,associated_labels,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,interest,language_code,listens.1,lyricist,number,publisher,title.1,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94
0,2,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,168,2,Hip-Hop,[21],[21],4656,en,1293,NaN,3,NaN,Food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,medium,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,237,1,Hip-Hop,[21],[21],1470,en,514,NaN,4,NaN,Electric Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,206,6,Hip-Hop,[21],[21],1933,en,1151,NaN,6,NaN,This World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0,11/26/08 1:45,2/6/08 0:00,NaN,4,6,47632,NaN,Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...",3,11/26/08 1:42,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,training,small,192000,0,Kurt Vile,11/25/08 17:49,11/26/08 0:00,161,178,Pop,[10],[10],54881,en,50135,NaN,1,NaN,Freeway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:48,1/1/08 0:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",978,en,361,NaN,3,NaN,Spiritual Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
tracks.dropna

<bound method DataFrame.dropna of    track_id comments   date_created date_released engineer favorites  id  \
0         2        0  11/26/08 1:44   1/5/09 0:00      NaN         4   1   
1         3        0  11/26/08 1:44   1/5/09 0:00      NaN         4   1   
2         5        0  11/26/08 1:44   1/5/09 0:00      NaN         4   1   
3        10        0  11/26/08 1:45   2/6/08 0:00      NaN         4   6   
4        20        0  11/26/08 1:45   1/6/09 0:00      NaN         2   4   
5        26        0  11/26/08 1:45   1/6/09 0:00      NaN         2   4   
6        30        0  11/26/08 1:45   1/6/09 0:00      NaN         2   4   
7        46        0  11/26/08 1:45   1/6/09 0:00      NaN         2   4   
8        48        0  11/26/08 1:45   1/6/09 0:00      NaN         2   4   
9       134        0  11/26/08 1:44   1/5/09 0:00      NaN         4   1   
10      135        1  11/26/08 1:49   1/7/09 0:00      NaN         0  58   
11      136        1  11/26/08 1:49   1/7/09 0:00     

In [11]:
tracks.head()

,track_id,comments,date_created,date_released,engineer,favorites,id,listens,producer,title,tracks,type,active_year_begin,active_year_end,associated_labels,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,interest,language_code,listens.1,lyricist,number,publisher,title.1,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94
0,2,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,168,2,Hip-Hop,[21],[21],4656,en,1293,NaN,3,NaN,Food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,medium,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,237,1,Hip-Hop,[21],[21],1470,en,514,NaN,4,NaN,Electric Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,206,6,Hip-Hop,[21],[21],1933,en,1151,NaN,6,NaN,This World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0,11/26/08 1:45,2/6/08 0:00,NaN,4,6,47632,NaN,Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...",3,11/26/08 1:42,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,training,small,192000,0,Kurt Vile,11/25/08 17:49,11/26/08 0:00,161,178,Pop,[10],[10],54881,en,50135,NaN,1,NaN,Freeway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:48,1/1/08 0:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",978,en,361,NaN,3,NaN,Spiritual Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
tracks = tracks.drop(tracks.columns[44:],1)

In [15]:
tracks

,track_id,comments,date_created,date_released,engineer,favorites,id,listens,producer,title,tracks,type,active_year_begin,active_year_end,associated_labels,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genre_top,genres,genres_all,interest,language_code,listens.1,lyricist,number,publisher,title.1
0,2,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,168,2,Hip-Hop,[21],[21],4656,en,1293,NaN,3,NaN,Food
1,3,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,medium,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,237,1,Hip-Hop,[21],[21],1470,en,514,NaN,4,NaN,Electric Ave
2,5,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,small,256000,0,NaN,11/26/08 1:48,11/26/08 0:00,206,6,Hip-Hop,[21],[21],1933,en,1151,NaN,6,NaN,This World
3,10,0,11/26/08 1:45,2/6/08 0:00,NaN,4,6,47632,NaN,Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...",3,11/26/08 1:42,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,training,small,192000,0,Kurt Vile,11/25/08 17:49,11/26/08 0:00,161,178,Pop,[10],[10],54881,en,50135,NaN,1,NaN,Freeway
4,20,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:48,1/1/08 0:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",978,en,361,NaN,3,NaN,Spiritual Level
5,26,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:49,1/1/08 0:00,181,0,NaN,"[76, 103]","[17, 10, 76, 103]",1060,en,193,NaN,4,NaN,Where is your Love?
6,30,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:49,1/1/08 0:00,174,0,NaN,"[76, 103]","[17, 10, 76, 103]",718,en,612,NaN,5,NaN,Too Happy
7,46,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:49,1/1/08 0:00,104,0,NaN,"[76, 103]","[17, 10, 76, 103]",252,en,171,NaN,8,NaN,Yosemite
8,48,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,2710,NaN,Niris,13,Album,1/1/90 0:00,1/1/11 0:00,NaN,2,11/26/08 1:42,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,training,large,256000,0,NaN,11/26/08 1:49,1/1/08 0:00,205,0,NaN,"[76, 103]","[17, 10, 76, 103]",247,en,173,NaN,9,NaN,Light of Light
9,134,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,6073,NaN,AWOL - A Way Of Life,7,Album,1/1/06 0:00,NaN,NaN,0,11/26/08 1:42,9,1,40.0583238,New Jersey,-74.4057,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,training,medium,256000,0,NaN,11/26/08 1:43,11/26/08 0:00,207,3,Hip-Hop,[21],[21],1126,en,943,NaN,5,NaN,Street Music


In [29]:
feats = ['tracks','favorites','comments.1','favorites.2','listens.1','interest','duration','comments.2','bit_rate']

X = tracks[feats]
y = tracks.genre_top.values

In [31]:
#y.fillna('Mac_Scheffer_Music', inplace=True)

AttributeError: 'numpy.ndarray' object has no attribute 'fillna'

In [33]:
#y.isna().sum()

In [34]:
len(y.unique())

d = {}

for i in range(0,7):
    d[list(y.unique())[i]] = i

print(d)

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [25]:
y = y.map(d)

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=42)
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/anaconda2/envs/py3k/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda2/envs/py3k/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [38]:
X_test

,tracks,favorites,comments.1,favorites.2,listens.1,interest,duration,comments.2,bit_rate
39,13,0,0,1,140,159,185,0,192000
25,2,0,0,0,148,701,192,0,192000
26,2,0,0,0,98,637,193,0,192000
43,4,0,0,1,270,815,144,0,192000
35,13,0,0,0,121,175,147,0,192000
41,13,0,0,1,80,149,278,0,192000
4,13,2,2,0,361,978,311,0,256000
12,2,2,0,2,1278,2559,1233,0,256000
8,13,2,2,0,173,247,205,0,256000
3,2,4,3,178,50135,54881,161,0,192000


In [41]:
log_reg.score(X_test, y_test)

0.36363636363636365

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.